----

# CMSE401 Quiz Instructions

This quiz is designed to take approximately 30 minutes to complete (you will be given the entire class period).  

Please read the following instructions before starting the quiz.


> This is an open Internet quiz.  Feel free to use anything on the Internet with one important exception...
> 
> - **DO NOT** communicate live with other people or AI tools during the quiz (either verbally or online).  The goal here is to find answers to problems as you would in the real world and demonstrate your own ability to solve problems.  
> 
> You will be given **until the end of class** to complete this quiz.  Use your time wisely. 
> 
> **HINTS:**
> - Neatness and grammar are important.  We will ignore all notes or code we can not read or understand.
> - Read the entire quiz from beginning to end before starting.  Not all questions are equal in **points vs. time** so plan your time accordingly. 
> - Spaces for answers are provided. Delete the prompting text such as "Put your answer to the above question here" and replace it with your answer. Do not leave the prompting text with your answer.
> - Do not assume that the answer must be in the same format of the cell provided. Feel free to change the cell formatting (e.g., markdown to code, and vice versa) or add additional cells as needed to provide your answer.
> - When a question asks for an answer "**in your own words**" it is still okay to search the Internet for the answer as a reminder. *However*, we would like you to do more than cut and paste.  Make the answer your own. 
> - If you get stuck, try not to leave an answer blank. It is better to include some notes or stub functions so we have an idea about your thinking process so we can give you partial credit.   
> - Always provid links to any references you find helpful. 
> - Feel free to delete the provided check marks (&#9989;) as a way to keep track of which questions you have successfully completed. 

> **Honor Code**
> 
> I, agree to neither give nor receive any help on this quiz from other people.  I also understand that providing answers to questions on this quiz to other students is also an academic misconduct violation as is live communication or receiving answers to questions on this quiz from other people. It is important to me to be a person of integrity and that means that ALL ANSWERS on this quiz are my answers.
> 
> &#9989; **<font color=red>DO THIS:</font>** Include your name in the line below to acknowledge the above statement:

Justin Wijaya

---

# Simple MPI Code

The following MPI code is intended to send messages between two MPI threads, BUT there is a bug with this code that you will need to explain and fix. 

```c
#include <stdio.h>
#include <mpi.h>

int main(int argc, char **argv) {
    int rank, size;
    int msg1;
    int msg2;
    MPI_Status status;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);


    if (rank == 0) {
        msg1=rank;
        MPI_Recv(&msg2, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &status);
        MPI_Send(&msg1, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
    } else if (rank == 1) {
        msg2=rank;
        MPI_Recv(&msg1, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &status);
        MPI_Send(&msg2, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
    }
    if (rank == 0){
        printf("I am rank %d and received: %d \n",rank, msg2);
    } else if (rank == 1){
        printf("I am rank %d and received: %d \n",rank, msg1);
    }
    MPI_Finalize();
    return 0;
}

```

&#9989; **<font color=red>Question 1</font>**: (10 points) First, log into a dev node on the HPCC. Copy the buggy code into a C file and then compile and run the code, using number of threads = 2, to test it out. Write the code below that you used to compile and run the C code and then explain what you observed when trying to run it. Note: Feel free to run the code from a submission script if you are not comfortable with how to request multiple processes when launching an MPI job from the command line. If using a submission script, paste the contents here.  

```c
mpicc msg_mpi.c  // to compile
sbatch msg_mpi.sb  // to run
```
submission script:
```bash
#!/bin/bash --login
#SBATCH --time=00:01:00
#SBATCH -c 2
#SBATCH -N 10
#SBATCH --mem=40gb
module purge
module load OpenMPI
mpicc msg_mpi.c
mpirun ./a.out
```

&#9989; **<font color=red>Question 2</font>**: (10 points) Now that you have tested the code, explain here what is wrong with the code and why. 

Both ranks are waiting to receive a message from each other, thus nothing is printed. This is known as deadlock. (Sometimes I keep getting this annoying warning message recently about the powertools module which is what shows up in the slurm file, so please ignore that).

&#9989; **<font color=red>Question 3</font>**: (20 points) Now we want to fix the bug in the code. Modify the supplied code to fix the bug and then paste the fixed version here. 



```c
#include <stdio.h>
#include <mpi.h>

void checkError(int comm, int rank)
{
    if (comm != MPI_SUCCESS)
    {
        char error_string[MPI_MAX_ERROR_STRING];
        int length_of_error_string;
        MPI_Error_string(comm, error_string, &length_of_error_string);
        printf("Rank %d: Caught MPI error: %s\n", rank, error_string);
    }
}

int main(int argc, char **argv) {
    int rank, size, err_code;
    int msg1;
    int msg2;
    MPI_Status status;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    MPI_Comm_set_errhandler(MPI_COMM_WORLD, MPI_ERRORS_RETURN);
    if (rank == 0) {
        msg1=rank;
        err_code=MPI_Send(&msg1, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        checkError(err_code, rank);
        err_code=MPI_Recv(&msg2, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &status);
        checkError(err_code, rank);
    } else if (rank == 1) {
        msg2=rank;

        err_code=MPI_Send(&msg2, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
        checkError(err_code, rank);
        err_code=MPI_Recv(&msg1, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &status);
        checkError(err_code, rank);
    }
    if (rank == 0){
        printf("I am rank %d and received: %d \n",rank, msg2);
    } else if (rank == 1){
        printf("I am rank %d and received: %d \n",rank, msg1);
    }
    MPI_Finalize();
    return 0;
}
```

&#9989; **<font color=red>Question 4</font>**: (20 points) Now lets update the code to use a third process. Update the fixed code so that rank 0 sends a message to rank 1, rank 1 sends a message to rank 2, and rank 2 sends a message to rank 0. Once the communication is complete, have each rank print out the message it recieved along with its own rank ID. Confirm the code works and paste the output along with the code below. 


```c
#include <stdio.h>
#include <mpi.h>

void checkError(int comm, int rank)
{
    if (comm != MPI_SUCCESS)
    {
        char error_string[MPI_MAX_ERROR_STRING];
        int length_of_error_string;
        MPI_Error_string(comm, error_string, &length_of_error_string);
        printf("Rank %d: Caught MPI error: %s\n", rank, error_string);
    }
}

int main(int argc, char **argv) {
    int rank, size, err_code;
    int msg1;
    int msg2;
    int msg3;
    MPI_Status status;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    MPI_Comm_set_errhandler(MPI_COMM_WORLD, MPI_ERRORS_RETURN);
    if (rank == 0) {
        msg1=rank;
        err_code=MPI_Send(&msg1, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        checkError(err_code, rank);
        err_code=MPI_Recv(&msg3, 1, MPI_INT, 2, 0, MPI_COMM_WORLD, &status);
        checkError(err_code, rank);
    } else if (rank == 1) {
        msg2=rank;

        err_code=MPI_Send(&msg2, 1, MPI_INT, 2, 0, MPI_COMM_WORLD);
        checkError(err_code, rank);
        err_code=MPI_Recv(&msg1, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &status);
        checkError(err_code, rank);
    } else if (rank == 2){
        msg3=rank;

        err_code=MPI_Send(&msg3, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
        checkError(err_code, rank);
        err_code=MPI_Recv(&msg2, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &status);
        checkError(err_code, rank);
    }
    if (rank == 0){
        printf("I am rank %d and received: %d \n",rank, msg3);
    } else if (rank == 1){
        printf("I am rank %d and received: %d \n",rank, msg1);
    } else if (rank == 2){
        printf("I am rank %d and received: %d \n",rank, msg2);
    }
    MPI_Finalize();
    return 0;
}

```
I am rank 0 and received: 2 

I am rank 1 and received: 0

I am rank 2 and received: 1 

&#9989; **<font color=red>Question 5</font>**: (10 points) Explain what the default error handling approach is in MPI and then explain why someone might want to change to a different error hanlding strategy. 

The default error handling approach is to terminate the process once encounterd. Someone might want to change that in order to figure out what the error is so that they can attempt to fix their code.

# Congratulations

You are done with your quiz. Please save the file and upload the jupyter notebook and any other necessary files to the D2L dropbox. 

Written by Dr. Nathan Haut, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----